In [17]:
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import pickle
import ollama
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings


import pandas as pd
#from langchain_community.vectorstores import Chroma



import pickle
import pandas as pd
#rag_test_input_path='/home/matrix4284/MY_GEN_AI_PROJECTS/RAG/GraphRAG/graphrag-local-ollama/ragtest/input/'+file_name
import os
# importing shutil module
import shutil

from langchain_community.chat_models import ChatOllama

from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

#from sentence_transformers import SentenceTransformer
#embeddings = SentenceTransformer("all-MiniLM-L6-v2")

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import XMLOutputParser
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_text_splitters import CharacterTextSplitter
import pickle

In [19]:
pdf_enrichment_output_dir = '../pdf_enrichment/pdf_enriched_output/'  
pdf_enrichment_output_file = 'pdf_enriched_content_dict_phase5_extract_highligts_478_final.pickle'

with open(pdf_enrichment_output_dir+pdf_enrichment_output_file, 'rb') as handle:
    document_dict_deserialized = pickle.load(handle)

In [20]:
from langchain.document_loaders import PyPDFLoader
pdf_file="../pdf_parsing/pdf_source/Kubernetes_in_Action.pdf"
pdf_loader = PyPDFLoader(pdf_file)
pages = pdf_loader.load_and_split()
print(pages[2].page_content)

Kubernetes in Action
 www.allitebooks.com


In [21]:
full_doc_summary=[]
doc_len=len(document_dict_deserialized)
for i in range(0,doc_len):
    summary=document_dict_deserialized[i]["text"]
    full_doc_summary.append(summary)

In [22]:
full_doc_summary[11]

'68\nCHAPTER 3\nPods: running containers in Kubernetes\n(multiple copies of the same component will be deployed) and multiple versions or\nreleases (stable, beta, canary, and so on) will run concurrently. This can lead to hun-\ndreds of pods in the system. Without a mechanism for organizing them, you end up\nwith a big, incomprehensible mess, such as the one shown in figure 3.6. The figure\nshows pods of multiple microservices, with several running multiple replicas, and others\nrunning different releases of the same microservice.\nIt’s evident you need a way of organizing them into smaller groups based on arbitrary\ncriteria, so every developer and system administrator dealing with your system can eas-\nily see which pod is which. And you’ll want to operate on every pod belonging to a cer-\ntain group with a single action instead of having to perform the action for each pod\nindividually. \n Organizing pods and all other Kubernetes objects is done through labels.\n3.3.1\nIntroducing l

In [23]:
#LLM Model for Prompt Tuning
model_name="llama3.1"
llm = ChatOllama(base_url="http://192.168.50.100:11434",model=model_name,temperature=0.15)



In [26]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_text_splitters import CharacterTextSplitter
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain


question_prompt_template = """
                  Please provide a summary of the following text.
                  TEXT: {text}
                  SUMMARY:
                  """

question_prompt = PromptTemplate(
    template=question_prompt_template, input_variables=["text"]
)

refine_prompt_template = """
              Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
              """

refine_prompt = PromptTemplate(
    template=refine_prompt_template, input_variables=["text"])

In [30]:
refine_chain = load_summarize_chain(
    llm,
    chain_type="refine",
    question_prompt=question_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
)

In [34]:
from time import gmtime, strftime
start_time=strftime("%Y-%m-%d %H:%M:%S")

'2024-09-13 03:14:03'

In [ ]:
start_time=strftime("%Y-%m-%d %H:%M:%S")
print(start_time)
refine_outputs = refine_chain({"input_documents": pages})
end_time=strftime("%Y-%m-%d %H:%M:%S")
print(end_time)

2024-09-13 03:15:36


/home/matrix4284/mambaforge-pypy3/envs/search_agent_poc/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [31]:
import timeit